# Feature Engineering

Building user-level features from transaction, listening, and demographic data for the churn prediction model.

In [29]:
from google.cloud import bigquery
from dotenv import load_dotenv
import pandas as pd
import warnings
import os

warnings.filterwarnings('ignore')
load_dotenv()

GCP_PROJECT = os.environ.get("GCP_PROJECT")
client = bigquery.Client(project=GCP_PROJECT)

def run_query(sql):
    return client.query(sql).to_dataframe()

## Feature Categories

Based on the churn drivers analysis, features will be grouped into:

1. **Transaction features** - Auto-renew status, plan type, payment history
2. **Demographic features** - Age, gender, registration channel, tenure
3. **Listening features** - Activity levels, engagement patterns

## Transaction Features

Using the most recent transaction before the prediction cutoff (Jan 31, 2017).

In [30]:
# Transaction features from most recent transaction
run_query("""
WITH latest_transaction AS (
    SELECT 
        msno,
        payment_method_id,
        payment_plan_days,
        plan_list_price,
        actual_amount_paid,
        is_auto_renew,
        is_cancel,
        transaction_date,
        membership_expire_date,
        ROW_NUMBER() OVER (PARTITION BY msno ORDER BY transaction_date DESC) as rn
    FROM kkbox.transactions
    WHERE transaction_date <= 20170131
)
SELECT 
    msno,
    payment_method_id,
    payment_plan_days,
    plan_list_price,
    actual_amount_paid,
    is_auto_renew,
    is_cancel,
    transaction_date,
    membership_expire_date
FROM latest_transaction
WHERE rn = 1
LIMIT 5
""")

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,transaction_date,membership_expire_date
0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,22,395,1599,1599,0,0,20161023,20180206
1,++0+IdHga8fCSioOVpU8K7y4Asw8AveIApVH2r9q9yY=,32,410,1788,1788,0,0,20160704,20170821
2,++4YWSlKw9vXYEFQ1qrldiSvwW/rVVP+rhaxAUcT8sc=,38,410,1788,1788,0,0,20170120,20180306
3,++5HPICzApqAq9mYdGB/mdke0MbubM8yUKLOmrPt4xU=,13,195,894,894,0,0,20160927,20170410
4,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,90,298,298,0,0,20170131,20170504


Transaction features include payment plan, method, price, auto-renew status, cancellation status, etc.

In [31]:
# Count of historical transactions per user
run_query("""
SELECT 
    msno,
    COUNT(*) as transaction_count
FROM kkbox.transactions
WHERE transaction_date <= 20170131
GROUP BY msno
ORDER BY transaction_count DESC
LIMIT 10
""")

,msno,transaction_count
0,72gJqt1O31E/WoxAEYFn9LHNI6mAZFGera5Q6gvsFkA=,208
1,5ty4nZkq54z93wQtBN7RHVYj8rNghBDCVBH+3xmxf0I=,172
2,OGKDrZQDB3yewZhoSd5qqvmG5A1GcNTYMexO95NlH+g=,148
3,SNlFRAsmUqnXKPofSXA8WYUc5DtmLcUMy4pXSJ3Ohz0=,131
4,vf6eQrnFfiS9o1kB/gRUJ4iTUixS9tHNKizqQ/1vLDI=,117
5,hQirtFbRr8t5T+/aVbrXM2tRNvoOIzChx5qzm1v/PE0=,104
6,IO+048XfFoL7X++ftIyb/zFKwNUUAkKaZLWrvg48TXA=,89
7,gzTushiuaWZwJShNepFWD+O9a7C76pAfW7Vc07jHXw8=,89
8,cmlCHjCLeo0TQK1S/37OqQK1AvwJWkD2h5HBd9lATgw=,78
9,F9//z42OpKq9GUSUNuSWmKIIww4deT2elqiqfSxznK0=,76


Some users have over 100 transactions in the 2-year window. Transaction count reflects tenure and engagement with the subscription system.

## Demographic Features

In [32]:
# Demographic features with tenure calculation
run_query("""
SELECT 
    msno,
    city,
    CASE WHEN bd BETWEEN 13 AND 100 THEN bd ELSE NULL END as age,
    gender,
    registered_via,
    registration_init_time,
    DATE_DIFF(
        DATE '2017-01-31',
        PARSE_DATE('%Y%m%d', CAST(registration_init_time AS STRING)),
        DAY
    ) as tenure_days
FROM kkbox.members
LIMIT 5
""")

,msno,city,age,gender,registered_via,registration_init_time,tenure_days
0,R9JIX15dpdKxvaPkoSft1gP1THFNxIaBwhcHiXjyMOg=,1,<NA>,None,9,20050608,4255
1,3VJyY4WUmeImm2EAFGLvtiVQU+3mwNgvmHT6SQS/1wg=,1,<NA>,None,9,20050629,4234
2,92aj7p/zkkOCGmfpn/V4j9wwoXI4z5VL1RS/IFx+VG4=,1,<NA>,None,9,20050822,4180
3,cs/1d0gFPWwSwjhxAElWKrkOajG9vvaySPsPRQ2PLN0=,5,<NA>,None,9,20060120,4029
4,S6BEuKNz1N73Idacuwpoi56ns7fdlm6wAGGO5JCx41g=,1,<NA>,None,9,20060211,4007


Tenure calculation works. Some users registered over 10 years ago (4000+ days). Age shows NA for invalid values as expected.

## Listening Features

Aggregate listening behavior from the last 30 days (Jan 2017) and overall history.

**Data truncation note:** The user_logs and transactions tables start January 1, 2015. For users who registered earlier, "total" metrics are truncated. To make features comparable, raw totals are normalized by observable_months (months between max(registration_date, 2015-01-01) and 2017-01-31), converting counts to per-month rates.

In [33]:
# Last 30 days listening features
run_query("""
SELECT 
    msno,
    COUNT(*) as days_active_last30,
    SUM(num_25) as songs_25_last30,
    SUM(num_100) as songs_100_last30,
    SUM(num_unq) as unique_songs_last30,
    ROUND(SUM(total_secs) / 3600, 1) as hours_listened_last30
FROM kkbox.user_logs
WHERE date >= 20170101 AND date <= 20170131
  AND total_secs >= 0 AND total_secs <= 57600
GROUP BY msno
LIMIT 5
""")

,msno,days_active_last30,songs_25_last30,songs_100_last30,unique_songs_last30,hours_listened_last30
0,DovoJJc380wzoTHPqqm1wVFpjPuoP2YlaRehJBJg9SM=,31,134,649,687,42.8
1,A74Obikrk6dRde7OKkZFgHcSy/aVJJ85lXH71AzR7Tc=,20,24,359,393,27.7
2,yOvSvbTnpr3ekeMs/UvBwAJBccJRRdAb8+CMBdJiQaU=,6,15,90,104,6.2
3,7t5G6ZXsxb18YN7Hq/FEiAnJ5Vmob92G5lu7t8+htW4=,14,57,628,692,47.9
4,AJ+xiXmqLRn6gnK6/PhOgn6AwWVjy2UCtMSqx/VC2Kg=,26,154,781,869,55.1


## Combined Feature Table

In [34]:
# Create full feature table
feature_query = """
WITH latest_transaction AS (
    SELECT
        msno,
        payment_method_id,
        payment_plan_days,
        plan_list_price,
        actual_amount_paid,
        is_auto_renew,
        is_cancel,
        transaction_date,
        membership_expire_date,
        ROW_NUMBER() OVER (PARTITION BY msno ORDER BY transaction_date DESC) as rn
    FROM kkbox.transactions
),
transaction_counts AS (
    SELECT
        msno,
        COUNT(*) as transaction_count
    FROM kkbox.transactions
    GROUP BY msno
),
listening_last30 AS (
    SELECT
        msno,
        COUNT(*) as days_active_last30,
        SUM(num_25) as songs_25_last30,
        SUM(num_100) as songs_100_last30,
        SUM(num_unq) as unique_songs_last30,
        ROUND(SUM(total_secs) / 3600, 1) as hours_listened_last30
    FROM kkbox.user_logs
    WHERE date >= 20170101 AND date <= 20170131
      AND total_secs >= 0 AND total_secs <= 57600
    GROUP BY msno
),
listening_total AS (
    SELECT
        msno,
        COUNT(*) as total_days_active,
        SUM(num_100) as total_songs_completed,
        ROUND(SUM(total_secs) / 3600, 1) as total_hours_listened
    FROM kkbox.user_logs
    WHERE total_secs >= 0 AND total_secs <= 57600
    GROUP BY msno
),
member_info AS (
    SELECT
        msno,
        city,
        CASE WHEN bd BETWEEN 13 AND 100 THEN bd ELSE NULL END as age,
        gender,
        registered_via,
        CASE
            WHEN registration_init_time > 20170131 THEN NULL
            ELSE DATE_DIFF(
                DATE '2017-01-31',
                PARSE_DATE('%Y%m%d', CAST(registration_init_time AS STRING)),
                DAY
            )
        END as tenure_days,
        COALESCE(
            CASE
                WHEN registration_init_time > 20170131 THEN NULL
                ELSE DATE_DIFF(
                    DATE '2017-01-31',
                    GREATEST(
                        PARSE_DATE('%Y%m%d', CAST(registration_init_time AS STRING)),
                        DATE '2015-01-01'
                    ),
                    MONTH
                ) + 1
            END,
            25
        ) as observable_months
    FROM kkbox.members
),
imputation_stats AS (
    SELECT
        APPROX_QUANTILES(m.age, 2)[OFFSET(1)] as median_age,
        APPROX_QUANTILES(m.tenure_days, 2)[OFFSET(1)] as median_tenure
    FROM kkbox.train t
    INNER JOIN latest_transaction lt ON t.msno = lt.msno AND lt.rn = 1
    LEFT JOIN member_info m ON t.msno = m.msno
)
SELECT
    t.msno,
    t.is_churn,
    -- Transaction features
    lt.payment_method_id,
    lt.payment_plan_days,
    lt.plan_list_price,
    lt.actual_amount_paid,
    lt.plan_list_price - lt.actual_amount_paid as discount,
    lt.is_auto_renew,
    lt.is_cancel,
    -- Demographic features
    CAST(COALESCE(m.city, -1) AS INT64) as city,
    CAST(COALESCE(m.age, s.median_age) AS INT64) as age,
    COALESCE(m.gender, 'unknown') as gender,
    CAST(COALESCE(m.registered_via, -1) AS INT64) as registered_via,
    CAST(COALESCE(m.tenure_days, s.median_tenure) AS INT64) as tenure_days,
    -- Observable months for normalization (data starts 2015-01-01)
    COALESCE(m.observable_months, 25) as observable_months,
    -- Listening features (last 30 days)
    COALESCE(l30.days_active_last30, 0) as days_active_last30,
    COALESCE(l30.songs_25_last30, 0) as songs_skipped_last30,
    COALESCE(l30.songs_100_last30, 0) as songs_completed_last30,
    COALESCE(l30.unique_songs_last30, 0) as unique_songs_last30,
    COALESCE(l30.hours_listened_last30, 0) as hours_listened_last30,
    -- Normalized features (per-month rates)
    ROUND(COALESCE(tc.transaction_count, 0) * 1.0 / COALESCE(m.observable_months, 25), 2) as transactions_per_month,
    ROUND(COALESCE(ltot.total_days_active, 0) * 1.0 / COALESCE(m.observable_months, 25), 2) as days_active_per_month,
    ROUND(COALESCE(ltot.total_songs_completed, 0) * 1.0 / COALESCE(m.observable_months, 25), 1) as songs_completed_per_month,
    ROUND(COALESCE(ltot.total_hours_listened, 0) / COALESCE(m.observable_months, 25), 2) as hours_listened_per_month
FROM kkbox.train t
INNER JOIN latest_transaction lt ON t.msno = lt.msno AND lt.rn = 1
CROSS JOIN imputation_stats s
LEFT JOIN transaction_counts tc ON t.msno = tc.msno
LEFT JOIN member_info m ON t.msno = m.msno
LEFT JOIN listening_last30 l30 ON t.msno = l30.msno
LEFT JOIN listening_total ltot ON t.msno = ltot.msno
"""

features_df = run_query(feature_query)
print(f"Shape: {features_df.shape}")
print(f"Users with transaction data: {len(features_df):,}")
print(f"Churn rate: {features_df['is_churn'].mean()*100:.1f}%")
features_df.isnull().sum()

Shape: (933578, 24)
Users with transaction data: 933,578
Churn rate: 6.2%


msno                         0
is_churn                     0
payment_method_id            0
payment_plan_days            0
plan_list_price              0
actual_amount_paid           0
discount                     0
is_auto_renew                0
is_cancel                    0
city                         0
age                          0
gender                       0
registered_via               0
tenure_days                  0
observable_months            0
days_active_last30           0
songs_skipped_last30         0
songs_completed_last30       0
unique_songs_last30          0
hours_listened_last30        0
transactions_per_month       0
days_active_per_month        0
songs_completed_per_month    0
hours_listened_per_month     0
dtype: int64

In [35]:
# Verify row counts and missing values
features_model = features_df.copy()
total_train_users = run_query("SELECT COUNT(*) as n FROM kkbox.train")['n'].iloc[0]
excluded_count = total_train_users - len(features_model)

print(f"Users for modeling: {len(features_model):,} ({len(features_model)/total_train_users*100:.1f}%)")
print(f"Excluded (no transactions): {excluded_count:,}")
print(f"\nMissing values:")
print(features_model.isnull().sum())

Users for modeling: 933,578 (96.1%)
Excluded (no transactions): 37,382

Missing values:
msno                         0
is_churn                     0
payment_method_id            0
payment_plan_days            0
plan_list_price              0
actual_amount_paid           0
discount                     0
is_auto_renew                0
is_cancel                    0
city                         0
age                          0
gender                       0
registered_via               0
tenure_days                  0
observable_months            0
days_active_last30           0
songs_skipped_last30         0
songs_completed_last30       0
unique_songs_last30          0
hours_listened_last30        0
transactions_per_month       0
days_active_per_month        0
songs_completed_per_month    0
hours_listened_per_month     0
dtype: int64


933,578 users (96.1%) have transaction data for modeling. Missing values are handled in the query:
- City and registered_via: -1 for missing
- Gender: 'unknown' for missing
- Age: median imputed (28 years)
- Tenure: median imputed (1,026 days)

In [36]:
# Check churn rate in excluded vs included users
excluded_churn = run_query("""
WITH has_transaction AS (
    SELECT DISTINCT msno FROM kkbox.transactions
)
SELECT 
    ROUND(AVG(t.is_churn) * 100, 1) as excluded_churn_rate
FROM kkbox.train t
LEFT JOIN has_transaction ht ON t.msno = ht.msno
WHERE ht.msno IS NULL
""")

print("Churn rate comparison:")
print(f"Included users: {features_model['is_churn'].mean()*100:.1f}%")
print(f"Excluded users: {excluded_churn['excluded_churn_rate'].iloc[0]}%")

Churn rate comparison:
Included users: 6.2%
Excluded users: 78.7%


Excluded users churn at 78.7% vs 6.2% for included users. Users without transaction records likely had subscriptions that lapsed without renewal.

The model predicts churn among active subscribers, not users who have already disengaged from the payment process.

## Verify Data Quality

In [37]:
# Verify data types
print("Data types:")
print(features_model.dtypes)
print(f"\nTotal missing values: {features_model.isnull().sum().sum()}")

Data types:
msno                          object
is_churn                       Int64
payment_method_id              Int64
payment_plan_days              Int64
plan_list_price                Int64
actual_amount_paid             Int64
discount                       Int64
is_auto_renew                  Int64
is_cancel                      Int64
city                           Int64
age                            Int64
gender                        object
registered_via                 Int64
tenure_days                    Int64
observable_months              Int64
days_active_last30             Int64
songs_skipped_last30           Int64
songs_completed_last30         Int64
unique_songs_last30            Int64
hours_listened_last30        float64
transactions_per_month       float64
days_active_per_month        float64
songs_completed_per_month    float64
hours_listened_per_month     float64
dtype: object

Total missing values: 0


Missing value handling:
- City and registered_via: -1 for missing
- Gender: 'unknown' for missing
- Age: median (28 years)
- Tenure: median (1,026 days)

## Export Features for Modeling

In [38]:
# Final feature set summary
print(f"Final modeling dataset: {len(features_model):,} users")
print(f"Churn rate: {features_model['is_churn'].mean()*100:.1f}%")
print(f"\nFeatures: {features_model.shape[1] - 2} (excluding msno and is_churn)")
print(features_model.dtypes)

Final modeling dataset: 933,578 users
Churn rate: 6.2%

Features: 22 (excluding msno and is_churn)
msno                          object
is_churn                       Int64
payment_method_id              Int64
payment_plan_days              Int64
plan_list_price                Int64
actual_amount_paid             Int64
discount                       Int64
is_auto_renew                  Int64
is_cancel                      Int64
city                           Int64
age                            Int64
gender                        object
registered_via                 Int64
tenure_days                    Int64
observable_months              Int64
days_active_last30             Int64
songs_skipped_last30           Int64
songs_completed_last30         Int64
unique_songs_last30            Int64
hours_listened_last30        float64
transactions_per_month       float64
days_active_per_month        float64
songs_completed_per_month    float64
hours_listened_per_month     float64
dtype: object

In [39]:
# Save to CSV for modeling notebook
features_model.to_csv('../data/features_for_modeling.csv', index=False)
print("Saved ../data/features_for_modeling.csv")

Saved ../data/features_for_modeling.csv


## Summary

**Output:** 933,578 users (96.1% of train set) with 22 features for modeling

**Features by category:**
- Transaction (7): payment method, plan days, list price, amount paid, discount, auto-renew, cancel
- Demographic (6): city, age, gender, registered_via, tenure_days, observable_months
- Listening last-30-days (5): days active, songs skipped, songs completed, unique songs, hours
- Normalized rates (4): transactions, days active, songs, hours per observable month

**Key design decisions:**
- Raw totals normalized by observable_months since data starts 2015-01-01
- Missing demographics: -1 (categorical), 'unknown' (gender), median (age/tenure)

**Selection bias note:** Excluded users (no transaction records) have 78.7% churn vs 6.2% for included users. The model predicts churn among active subscribers, not users who already disengaged.